In [1]:
import subprocess
def get_html_from_url(url):
    cmds = ["/Applications/Google Chrome.app/Contents/MacOS/Google Chrome", "--headless", "--dump-dom", "--enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb"]
    cmds.append(url)
    result = subprocess.run(cmds, stdout=subprocess.PIPE)
    text = result.stdout.decode('utf-8')
    return text
    
# t = get_html_from_url("https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:systems-of-equations/x2f8bb11595b61c86:introduction-to-systems-of-equations/v/trolls-tolls-and-systems-of-equation")
# t
# # result = subprocess.run(['pwd'], stdout=subprocess.PIPE)
# # text = result.stdout.decode('utf-8')
# # text


In [16]:
# unit_url = "https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:systems-of-equations/x2f8bb11595b61c86:introduction-to-systems-of-equations"
unit_url = "https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:quadratics-multiplying-factoring"
site = "https://www.khanacademy.org"

In [17]:
from xml.dom.minidom import parseString

html = get_html_from_url(unit_url)



In [18]:
from html.parser import HTMLParser
class Parser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.href = ''
        self.urls = []

    def handle_starttag(self, tag, attrs):
        if (tag == 'a'):
            for (k,v) in attrs:
                if (k == 'href'):
                    self.href = v
        elif (tag == 'span'):
            for (k,v) in attrs:
                if (k == 'aria-label' and v == 'Video'):
                    self.urls.append(self.href)            



parser = Parser()
parser.feed(html)
parser.urls


['/math/algebra/x2f8bb11595b61c86:quadratics-multiplying-factoring/x2f8bb11595b61c86:multiply-monomial-polynomial/v/polynomials-intro',
 '/math/algebra/x2f8bb11595b61c86:quadratics-multiplying-factoring/x2f8bb11595b61c86:multiply-monomial-polynomial/v/multiply-monomials-polynomials-area-model',
 '/math/algebra/x2f8bb11595b61c86:quadratics-multiplying-factoring/x2f8bb11595b61c86:multiply-binomial/v/area-model-for-multiplying-binomials',
 '/math/algebra/x2f8bb11595b61c86:quadratics-multiplying-factoring/x2f8bb11595b61c86:multiply-binomial/v/multiplying-simple-binomials',
 '/math/algebra/x2f8bb11595b61c86:quadratics-multiplying-factoring/x2f8bb11595b61c86:multiply-binomial/v/multiplying-binomials',
 '/math/algebra/x2f8bb11595b61c86:quadratics-multiplying-factoring/x2f8bb11595b61c86:special-product-binomials/v/difference-of-squares-pattern-for-simple-binomials',
 '/math/algebra/x2f8bb11595b61c86:quadratics-multiplying-factoring/x2f8bb11595b61c86:special-product-binomials/v/pattern-for-squa

In [32]:
video_url = site + parser.urls[0]
video_html = get_html_from_url(video_url)




[0521/115618.386214:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0521/115618.527860:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.


In [49]:
class VideoParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.start_h1 = False
        self.start_div = False
        self.start_iframe = False
        
        self.name = ''
        self.desc = ''

    def handle_starttag(self, tag, attrs):
        if (tag == 'h1'):
            self.start_h1 = True       
        elif (tag == 'div'):
            for (k,v) in attrs:
                if (k == 'id' and v == 'ka-videoPageTabs-tabbedpanel-content'):
                    self.start_div = True
        elif (tag == 'iframe'):
            print(attrs)
            for (k,v) in attrs:
                if(k == 'data-youtubeid'):
                    print(v)

    def handle_endtag(self, tag):
        if (tag == 'h1'):
            self.start_h1 = False
        elif (tag == 'div'):
            self.start_div = False
            
    def handle_data(self, data):
        if self.start_h1:
            self.name = data.title()
        if self.start_div:
            self.desc += data

video_parser = VideoParser()
video_parser.feed(video_html)
print(video_parser.name, video_parser.desc)




[('src', 'https://www.googletagmanager.com/ns.html?id=GTM-WSX5PBP'), ('height', '0'), ('width', '0'), ('style', 'display:none;visibility:hidden')]
Systems Of Equations: Trolls, Tolls (1 Of 2) A troll forces us to use algebra to figure out the make-up of his currency. We end up setting up a system of equations. Created by Sal Khan.


In [5]:
def get_searched_value(video_html, searchStr):
    x = video_html.find(searchStr)
    start = video_html.find('"', x+len(searchStr))
    end = video_html.find('"', start+1)
    return video_html[start+1:end]


def get_topic(video_html):
    desc =  get_searched_value(video_html, '"translatedDescription":')
    name = get_searched_value(video_html, '"translatedTitle":').title()
    videoID = get_searched_value(video_html, '"translatedYoutubeId":')
    return {
        "name": name,
        "description": desc,
        "texts": ["text1.md"],
        "video_parts": [{
            "id": videoID
        }],
        "node_type": "lecture",
        "parent_id": "khan-algebra-1",
        "thumbnail": ""
    }

In [19]:
import json
topics = []
for url in parser.urls:
    video_url = site + url
    video_html = get_html_from_url(video_url)
    topic = get_topic(video_html)
    print(json.dumps(topic, indent=4))
    topics.append(topic)
    


    
    
    
    

{
    "name": "Polynomials Intro",
    "description": "This introduction to polynomials\u00a0covers common terminology like terms, degree, standard form, monomial, binomial and trinomial. Polynomials are sums of terms of the form k\u22c5x\u207f, where k is any number and n is a positive integer. For example, 3x+2x-5 is a polynomial.\u00a0",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "Vm7H0VTlIco"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Multiply Monomials By Polynomials: Area Model",
    "description": "We can use an area model to multiply a polynomial by a single term.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "wznE1wlmhR0"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Multiplying Binomials: Area Model",
    "description": "Sal expresses the are

{
    "name": "Factoring Difference Of Squares: Leading Coefficient \u2260 1",
    "description": "Sal factors 45x^2-125 as 5(3x+5)(3x-5).",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "jmbg-DKWuc4"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Factoring Difference Of Squares: Analyzing Factorization",
    "description": "Sal analyzes two different factorizations of 16x^2-64 and determines whether they are correct.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "TK-U1p9O6Nc"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Factoring Difference Of Squares: Shared Factors",
    "description": "Sal finds the binomial factor shared by m^2-4m-45 and 6m^2-150.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "xYQR2LNZLeU"
 